In [187]:
# Initial imports
import os
import pandas as pd
from dotenv import load_dotenv
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
from newsapi import NewsApiClient
from finvizfinance.screener.overview import Overview
import alpaca_trade_api as tradeapi
import datetime
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.tsa.arima_model import ARIMA
import arch
import math
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')


In [188]:
# Read your api key environment variable
from dotenv import load_dotenv
load_dotenv()
api_key = os.getenv("news_api")
print(api_key)

f147b186a1504298a5fca6370fdcfe78


In [189]:
# Create a newsapi client
newsapi = NewsApiClient(api_key=os.environ["news_api"])

In [190]:
foverview = Overview()

filters_dict = {
    'Performance':'Today +5%',
    'Current Volume':"Over 5M",
    'Country': 'USA'
}
foverview.set_filter(filters_dict=filters_dict)

tickers = foverview.ScreenerView()
#tickers.sort_values(by='Volume', inplace=True, ascending=False)
tickers.set_index('Ticker', inplace=True )
tickers.head(100)

,Company,Sector,Industry,Country,Market Cap,P/E,Price,Change,Volume
Ticker,,,,,,,,,
ACIC,Atlas Crest Investment Corp.,Financial,Shell Companies,USA,8.570000e+08,None,17.14,0.0646,15529439.0
ALTU,Altitude Acquisition Corp.,Financial,Shell Companies,USA,NaN,None,12.10,0.1330,5319302.0
AVGR,"Avinger, Inc.",Healthcare,Medical Devices,USA,2.036800e+08,None,2.10,0.2426,122519731.0
CEI,"Camber Energy, Inc.",Energy,Oil & Gas E&P,USA,5.611000e+07,None,2.23,0.0985,60393616.0
CLSK,"CleanSpark, Inc.",Technology,Software - Infrastructure,USA,9.467800e+08,None,38.10,0.2587,11558723.0
FE,FirstEnergy Corp.,Utilities,Utilities - Diversified,USA,1.851000e+10,30.36,34.25,0.0720,18822326.0
FLNT,"Fluent, Inc.",Communication Services,Advertising Agencies,USA,5.833300e+08,205.53,7.81,0.1468,7167900.0
FRX,Forest Road Acquisition Corp.,Financial,Shell Companies,USA,3.675000e+08,None,12.25,0.0987,7525275.0
GSAH,GS Acquisition Holdings Corp II,Financial,Shell Companies,USA,1.310000e+09,None,13.99,0.1237,12688881.0


In [191]:
def get_sentiment(company):
    stock_headlines = ( newsapi.get_everything(
    q=company,
    language='en',
    sort_by='relevancy'))

    sentiments = []

    for articles in stock_headlines['articles']:
        try:
            text = articles['content']
            results = analyzer.polarity_scores(text)
            compound = results['compound']
            pos = results['pos']
            neu = results['neu']
            neg = results['neg'] 

            #put data into sentiments
            sentiments.append({
                'text':text,
                'Compound':compound,
                'Positive':pos,
                'Negative':neg,
                'Neutral':neu})

        except AttributeError:
            pass
    fininal_sentiments = pd.DataFrame(sentiments)
    if not fininal_sentiments.empty:
        return(fininal_sentiments["Positive"].mean())
    else:
        return(0)

In [201]:
ticker_sentiment = [] 
for ticker in tickers.index: 
    sentiment = get_sentiment(tickers['Company'][ticker])
    print(sentiment)
    if float(sentiment) >= 0.1:
        ticker_sentiment.append(ticker)
        print(f"{ticker} meets our sentiment requirement")
    else:
        print(f"{ticker} does not meet our sentiment requirement")

NewsAPIException: {'status': 'error', 'code': 'rateLimited', 'message': 'You have made too many requests recently. Developer accounts are limited to 100 requests over a 24 hour period (50 requests available every 12 hours). Please upgrade to a paid plan if you need more requests.'}

In [202]:
### Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version = "v2",
    base_url = "https://paper-api.alpaca.markets"
)
account = api.get_account()
print(account.status)

DISABLE_PENDING


In [203]:
#Set time frame for reporting stocks
today = datetime.date.today()
tomorrow = today + datetime.timedelta(days = 2) 
start_time = tomorrow.strftime("%b %d %Y") + " 10:00AM"
next_day = today + datetime.timedelta(days = 1) 
end_time = next_day.strftime("%b %d %Y") + " 1:00PM"

date_from = datetime.datetime.strptime(
    start_time, '%b %d %Y %I:%M%p')
date_to = datetime.datetime.strptime(
    end_time, '%b %d %Y %I:%M%p')

In [204]:
# Set timeframe to '1D'
timeframe = "1D"

#Set timeframe for last 2 years
start_date = pd.Timestamp("2018-05-01", tz="America/New_York").isoformat()
end_date = pd.Timestamp(next_day, tz="America/New_York").isoformat()

#Get ticker data
df_ticker = api.get_barset(
    ticker_sentiment,
    timeframe,
    start=start_date,
    end=end_date
).df

df_ticker.dropna()
df_ticker.tail()


HTTPError: 403 Client Error: Forbidden for url: https://data.alpaca.markets/v1/bars/1D?symbols=&start=2018-05-01T00%3A00%3A00-04%3A00&end=2021-02-19T00%3A00%3A00-05%3A00

In [ ]:
df_closing_prices = pd.DataFrame()
for ticker in ticker_sentiment: 
    df_closing_prices[ticker] = df_ticker[ticker]["close"]

In [ ]:
df_closing_prices.dropna(inplace=True)
df_closing_prices.tail()

In [205]:
def hpfilter(ticker):
    ts_noise, ts_trend = sm.tsa.filters.hpfilter(df_ticker[ticker][['close']])

    #create different dataframes for each composition
    ticker_decomp = df_ticker[ticker][['close']].copy()
    ticker_decomp['noise'] = ts_noise
    ticker_decomp['trend'] = ts_trend
    return ts_noise.plot(title='Noise')

In [206]:
for ticker in ticker_sentiment: 
    print(hpfilter((ticker)))

In [207]:
price_forcasting = pd.DataFrame()
dfs=[]
for ticker in df_closing_prices: 
    model = ARIMA(df_closing_prices[ticker], order=(1,1,1))
    results = model.fit()
    df = pd.DataFrame(results.forecast(steps=15)[0], columns=[ticker])
    dfs.append(df)

final_df = pd.concat(dfs, axis=1)
final_df.head(20)

ValueError: No objects to concatenate

In [208]:
df_closing_list = df_closing_prices.iloc[[-1]].values.tolist()
final_df_list = final_df.iloc[[9]].values.tolist()


IndexError: positional indexers are out-of-bounds

In [209]:
base_price = df_closing_list[0]
enter_price = final_df_list[0]
print(enter_price,base_price)

[19.094860880867525, 2.4127369727329193, 13.431228235107623, 2.177651549798012] [17.25, 2.21, 13.98, 2.46]


In [210]:
investment = 2500

stock_capital = round(investment / len(stock_buys),2)
print(stock_capital)

1250.0


In [211]:

stock_buys=[]
number_of_shares = {}
num = 0 
for ticker in df_closing_prices: 
    value = round((enter_price[num] - base_price[num])/ base_price[num] * 100.0,2)
    if value > 7:
        stock_buys.append(ticker)
        number_of_shares[ticker] = math.floor(stock_capital/enter_price[num])
        api.submit_order(
            symbol=ticker,
            side='buy',
            type='limit',
            limit_price=enter_price[num],
            qty=number_of_shares[ticker],
            time_in_force='day',
            take_profit=dict(
                limit_price=enter_price[num]*1.50,
            ),
            stop_loss=dict(
                stop_price=enter_price[num]/1.20,
            )
        )
    num += 1


In [212]:
print(stock_buys)

[]


In [213]:
print(number_of_shares)

{}
